In [1]:
import subprocess
import os

# 설정
SIZE_THRESHOLD_MB = 5  # 5MB 이상
SHOW_TOP_N = 20

def get_large_git_files():
    print(f"\n📦 Git 이력 내 {SIZE_THRESHOLD_MB}MB 이상 파일 탐색 중...\n")

    # 모든 blob 목록 추출
    cmd_list = [
        "git", "rev-list", "--objects", "--all"
    ]
    rev_objects = subprocess.check_output(cmd_list).decode("utf-8").splitlines()

    cmd_batch = [
        "git", "cat-file", "--batch-check=%(objecttype) %(objectname) %(objectsize) %(rest)"
    ]
    batch_proc = subprocess.Popen(cmd_batch, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    input_data = "\n".join(line.split()[0] for line in rev_objects if line).encode("utf-8")
    stdout, _ = batch_proc.communicate(input=input_data)

    big_files = []
    for line in stdout.decode("utf-8").splitlines():
        if line.startswith("blob"):
            parts = line.split()
            size = int(parts[2])
            path = " ".join(parts[3:])
            if size >= SIZE_THRESHOLD_MB * 1024 * 1024:
                big_files.append((size, path))

    # 정렬 및 출력
    big_files.sort(reverse=True)
    print(f"{'Size (MB)':>10}  File path")
    print("=" * 60)
    for size, path in big_files[:SHOW_TOP_N]:
        print(f"{size / (1024 * 1024):10.2f}  {path}")

    return [path for _, path in big_files]


def append_to_gitignore(files):
    print("\n✏️  `.gitignore`에 추가합니다...\n")
    with open(".gitignore", "a") as f:
        for file in files:
            f.write(f"\n{file}")
            print(f"➕ {file}")
    print("\n✅ 완료!")

if __name__ == "__main__":
    files = get_large_git_files()
    if not files:
        print("❌ 큰 파일이 Git 이력에 없습니다.")
    else:
        add = input("\n👉 이 파일들을 `.gitignore`에 추가할까요? (y/n): ")
        if add.strip().lower() == "y":
            append_to_gitignore(files)



📦 Git 이력 내 5MB 이상 파일 탐색 중...



CalledProcessError: Command '['git', 'rev-list', '--objects', '--all']' returned non-zero exit status 128.